In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget

In [ ]:
plt.rcParams['figure.figsize'] = [9.6, 6.4]
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', size=9)
# plt.rc('legend', fontsize=7)
defaultColorCycle = plt.rcParams["axes.prop_cycle"]

# Emittance Growth in Drift Space

In [ ]:
plotDefs = [
    {
        'varName1': 'x', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'x',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'x', 'varName2': 'xp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'y', 'varName2': 'yp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'z', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'Ekin',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'z',
        'opacityHist': 0.5,
    },
]

## Load distribution showing emittance growth

In [ ]:
sourceFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/AllBuckets/RUN_2501_121416.2057.001'
distr1 = bd.convert_astra_to_standard_df(
    sourceFilePath, zProjection=None, zCut=None, saveStandardFwf=False, verbose=False
)
distr1.describe()

In [ ]:
plotDefs[0]['lims1'] = (-30, 30.)   # [mm]
plotDefs[0]['lims2'] = (-30., 30.)   # [mm]
plotDefs[1]['lims1'] = (115.024, 115.032)   # [ns]
plotDefs[1]['lims2'] = plotDefs[0]['lims1']
plotDefs[2]['lims1'] = plotDefs[1]['lims1']
plotDefs[2]['lims2'] = plotDefs[0]['lims2']
plotDefs[3]['lims1'] = plotDefs[0]['lims1']
plotDefs[3]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[4]['lims1'] = plotDefs[0]['lims2']
plotDefs[4]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[5]['lims1'] = (20.25e3, 20.65e3)   # [mm]
plotDefs[5]['lims2'] = (0., 650.)   # [MeV/c]
plotDefs[6]['lims1'] = plotDefs[1]['lims1']
plotDefs[6]['lims2'] = plotDefs[5]['lims2']   # [MeV]
plotDefs[7]['lims1'] = plotDefs[1]['lims1']
plotDefs[7]['lims2'] = plotDefs[5]['lims2']   # [MeV/c]
plotDefs[8]['lims1'] = plotDefs[1]['lims1']
plotDefs[8]['lims2'] = plotDefs[5]['lims1']

## Select single RF bucket

In [ ]:
filterSpecs = {
    'x': plotDefs[0]['lims1'],
    'y': plotDefs[0]['lims2'],
    'z': (20.55e3, 20.70e3),
    'px': plotDefs[3]['lims2'],
    'py': plotDefs[4]['lims2'],
    'pz': plotDefs[5]['lims2'],
    't': plotDefs[1]['lims1'],
}

In [ ]:
distr1Bucket = bd.filter_distr(distr1, filterSpecs)
distr1Bucket.describe()

In [ ]:
distr1Bucket.head()

## Load equivalent Gaussian beam

In [ ]:
sourceFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/LargeEspread/LargeEspread_z_distr.ini'
distr1GaussZ = bd.convert_astra_to_standard_df(
    sourceFilePath, zProjection=None, zCut=None, saveStandardFwf=False, verbose=False
)
distr1GaussZ.describe()

In [ ]:
sourceFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/LargeEspread/LargeEspread_t_distr.ini'
distr1GaussT = bd.convert_astra_to_standard_df(
    sourceFilePath, zProjection=None, zCut=None, saveStandardFwf=False, verbose=False
)
distr1GaussT.describe()

## Plot

In [ ]:
ax = bd.plot_distr(
    [distr1, distr1GaussZ, distr1GaussT, distr1Bucket], plotDefs,
    title='Astra output at z = 20.57 m'
)

## Save distribution with single RF bucket

### Full energy spread

In [ ]:
astraRefParticle = {
    'x': 0.,
    'px': 0.,
    'y': 0.,
    'py': 0.,
    'z': 20.600e3,
    'pz': 520.,
    't': 115.0275,
    'pdgId': -11,
}
astraRefParticle['Q'] = distr1Bucket['Q'][1]
distr1Bucket = distr1Bucket.append(astraRefParticle, ignore_index=True)
distr1Bucket.describe()

In [ ]:
distr1Bucket.tail()

In [ ]:
outFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/AllBuckets/RUN_2501_121416.2057.001'
outFilePath = outFilePath[:-4] + '_SingleBucket'
_ = bd.convert_standard_df_to_astra(distr1Bucket, refParticleId=distr1Bucket.shape[0]-1, outFilePath=outFilePath)

### Only larger energies

In [ ]:
filterSpecs['pz'] = (400., np.inf)   # [MeV/c]

In [ ]:
distr1BucketLargeE = bd.filter_distr(distr1Bucket, filterSpecs)
distr1BucketLargeE.describe()

In [ ]:
distr1BucketLargeE.tail()

In [ ]:
distr1Bucket.loc[distr1Bucket.shape[0]-1,:]

In [ ]:
outFilePath = outFilePath[:-4] + '_SingleBucket_LargeE'
_ = bd.convert_standard_df_to_astra(distr1BucketLargeE, refParticleId=distr1Bucket.shape[0]-1, outFilePath=outFilePath)

## Load output distribution

In [ ]:
sourceFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket/EmitGrowthInDriftSpace.2260.001'

In [ ]:
distrOutBucket = bd.convert_astra_to_standard_df(
    sourceFilePath, zProjection=None, zCut=None, saveStandardFwf=False, verbose=False
)
distrOutBucket.describe()

In [ ]:
distrOutBucketProj = bd.convert_astra_to_standard_df(
    sourceFilePath, zProjection=22.600e3, zCut=None, saveStandardFwf=False, verbose=False
)
distrOutBucketProj.describe()

In [ ]:
plotDefs[0]['lims1'] = (-30, 30.)   # [mm]
plotDefs[0]['lims2'] = (-30., 30.)   # [mm]
plotDefs[1]['lims1'] = (121.60, 121.78)   # [ns]
# plotDefs[1]['lims1'] = (148.28, 148.48)   # [ns]
plotDefs[1]['lims2'] = plotDefs[0]['lims1']
plotDefs[2]['lims1'] = plotDefs[1]['lims1']
plotDefs[2]['lims2'] = plotDefs[0]['lims2']
plotDefs[3]['lims1'] = plotDefs[0]['lims1']
plotDefs[3]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[4]['lims1'] = plotDefs[0]['lims2']
plotDefs[4]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[5]['lims1'] = (22.25e3, 22.65e3)   # [mm]
plotDefs[5]['lims2'] = (0., 600.)   # [MeV/c]
plotDefs[6]['lims1'] = plotDefs[1]['lims1']
plotDefs[6]['lims2'] = plotDefs[5]['lims2']   # [MeV]
plotDefs[7]['lims1'] = plotDefs[1]['lims1']
plotDefs[7]['lims2'] = plotDefs[5]['lims2']   # [MeV/c]
plotDefs[8]['lims1'] = plotDefs[1]['lims1']
plotDefs[8]['lims2'] = plotDefs[5]['lims1']

In [ ]:
ax = bd.plot_distr(
    [distrOutBucket, distrOutBucketProj], plotDefs,
    title='Astra output at z = 30.60 m'
)

## Emittance evolution along drift space

### Independent computation in Python

In [ ]:
zProjs = np.linspace(18.6, 30.6, 100) * 1e3   # [mm]
emitn = np.zeros(*zProjs.shape)
for ind, zProj in enumerate(zProjs):
    distrOutBucket = bd.convert_astra_to_standard_df(
        sourceFilePath, zProjection=zProj, zCut=None, saveStandardFwf=False, verbose=False
    )
    emitn[ind] = bd.compute_emittance(distrOutBucket, 'x', verbose=False)

### Computation in ASTRA (different variants)

In [ ]:
astraEmitX_LprojectEmitFalse = pd.read_fwf(
    '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket/EmitGrowthInDriftSpace.Xemit.002'
)
astraEmitX_LprojectEmitFalse.columns = ['z', 't', 'xAvg', 'xRms', 'xpRms', 'emitnX', 'xxpAvg']

In [ ]:
astraEmitX_LprojectEmitTrue = pd.read_fwf(
    '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket/EmitGrowthInDriftSpace.Xemit.001'
)
astraEmitX_LprojectEmitTrue.columns = ['z', 't', 'xAvg', 'xRms', 'xpRms', 'emitnX', 'xxpAvg']

In [ ]:
astraEmitX_LocalEmitTrue = pd.read_fwf(
    '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket/EmitGrowthInDriftSpace.Xemit.003'
)
astraEmitX_LocalEmitTrue.columns = ['z', 't', 'xAvg', 'xRms', 'xpRms', 'emitnX', 'xxpAvg']

In [ ]:
fig, ax = plt.subplots(figsize=(9.6, 6.4))
ax.plot(zProjs*1e-3, emitn, label='Python computation')
ax.plot(
    astraEmitX_LprojectEmitFalse['z'], astraEmitX_LprojectEmitFalse ['emitnX'],
    label='Astra computation, Lproject_emit=F'
)
ax.plot(
    astraEmitX_LprojectEmitTrue['z'], astraEmitX_LprojectEmitTrue['emitnX'],
    label='Astra computation, Lproject_emit=T'
)
ax.plot(
    astraEmitX_LocalEmitTrue['z'], astraEmitX_LocalEmitTrue['emitnX'],
    label='Astra computation, Local_emit=T'
)
ax.set_xlim([18.6, 30.6])
ax.set_ylim([10e3, 50e3])
ax.set_xlabel('z [m]')
ax.set_ylabel('emitn [pi mm mrad]')
ax.legend()
ax.grid()

### Evolution with different energy spreads

In [ ]:
astraEmitX_LprojectEmitTrue_LargeE = pd.read_fwf(
    '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket_LargeE/EmitGrowthInDriftSpace.Xemit.001'
)
astraEmitX_LprojectEmitTrue_LargeE.columns = ['z', 't', 'xAvg', 'xRms', 'xpRms', 'emitnX', 'xxpAvg']

In [ ]:
fig, ax = plt.subplots(figsize=(9.6, 6.4))
sourceFilePath = '/home/tia/tmp/EmitGrowthInDriftSpace/NicoDistr_Z20p57m/SingleBucket/EmitGrowthInDriftSpace.2260.001'
pzMins = np.array([550., 545., 540., 530., 520., 510., 500., 475., 450., 400., 350., 300., 200., 100., 50., 0.])   # [MeV/c]
colorSeq = plt.cm.viridis(np.linspace(0,1,pzMins.shape[0]))
for pzInd, pzMin in enumerate(pzMins,0):
    filterSpecs = {'pz': (pzMin, np.inf)}
    emitn = np.zeros(*zProjs.shape)
    for ind, zProj in enumerate(zProjs):
        distrOutBucket = bd.convert_astra_to_standard_df(
            sourceFilePath, zProjection=zProj, zCut=None, saveStandardFwf=False, verbose=False
        )
        emitn[ind] = bd.compute_emittance(distrOutBucket, 'x', filterSpecs=filterSpecs, verbose=False)
    ax.plot(zProjs*1e-3, emitn, '-', color=colorSeq[pzInd], label='pz > {:.1f} MeV/c'.format(pzMin))
ax.plot(
    astraEmitX_LprojectEmitTrue['z'], astraEmitX_LprojectEmitTrue['emitnX'],
    'o--', color='k', markerfacecolor=colorSeq[0], markeredgecolor=colorSeq[0], markevery=5,
    label='pz > 0 MeV/c, Astra output'
)
ax.plot(
    astraEmitX_LprojectEmitTrue_LargeE['z'], astraEmitX_LprojectEmitTrue_LargeE['emitnX'],
    'o--', color='k', markerfacecolor=colorSeq[8], markeredgecolor=colorSeq[8], markevery=5,
    label='pz > 400 MeV/c, Astra output'
)
ax.set_xlim([18.6, 30.6])
ax.set_ylim([10e3, 20e3])
ax.set_xlabel('z [m]')
ax.set_ylabel('emitn [pi mm mrad]')
ax.legend()
ax.grid()

## Emittance vs. energy range

In [ ]:
fig, ax = plt.subplots(figsize=(9.6, 6.4))
zProjs = np.array((20.6, 22.6, 24.6, 26.6)) * 1e3   # [m]
labelInds = np.delete(np.arange(len(pzMins)), [1, 2, 10, 12, 13, 14])
for zProj in zProjs:
    particleFractions = np.zeros(*pzMins.shape)
    emitn = np.zeros(*pzMins.shape)
    for pzInd, pzMin in enumerate(pzMins,0):
        filterSpecs = {'pz': (pzMin, np.inf)}
        distrOutBucket = bd.convert_astra_to_standard_df(
            sourceFilePath, zProjection=zProj, zCut=None, saveStandardFwf=False, verbose=False
        )
        distrSelected = bd.filter_distr(distrOutBucket, filterSpecs)
        particleFractions[pzInd] = distrSelected.shape[0] / distrOutBucket.shape[0]
        # _, particleFractions[pzInd] = bd.distr_within_ellipse(
        #     distrOutBucket, emitTraceSpace, ellipseSpecs)
        emitn[pzInd] = bd.compute_emittance(
            distrSelected, 'x', verbose=False
        )
    ax.plot(
        particleFractions, emitn,
        'o-', label='z = {:.1f} m'.format(zProj*1e-3)
    )
# ax.set_xscale('log')
ax.set_ylim([0, 18e3])
ax.set_xlabel('Beam fraction')
ax.set_ylabel('Norm. emittance [pi mm mrad]')
ax.legend()
ax.grid()

ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(particleFractions[labelInds])
ax2.set_xticklabels(['{:.0f}'.format(pzm) for pzm in pzMins[labelInds]])
_ = ax2.set_xlabel('Min. pz considered [Mev/c]')

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>